In [1]:
import numpy as np
from math import log2
from IPython.core.interactiveshell import InteractiveShell
import sys

In [2]:
def entropy(roomData):
    H=0.0
    i=0
    scoreboard = {1:0, 2:0, 3:0, 4:0}
    size = len(roomData)
    while i<size:
        if roomData[i][-1] == 1:
            scoreboard[1]+=1
        elif roomData[i][-1] == 2:
            scoreboard[2]+=1
        elif roomData[i][-1] == 3:
            scoreboard[3]+=1
        elif roomData[i][-1] == 4:
            scoreboard[4]+=1
        i+=1;
    for i in (1,2,3,4):
        if scoreboard[i] > 0:
            H += ((-scoreboard[i])/size)*(log2(scoreboard[i]/size))
    return H

def InfoGain(All, Left, Right):
    H_SAll=entropy(All)
    Total=len(Left)+len(Right)
    Remainder=((len(Left)/Total)*entropy(Left))+((len(Right)/Total)*entropy(Right))
    Gains=H_SAll-Remainder
    return Gains

def FIND_SPLIT(d):
    emitter = 0
    value = 0
    max_info_gain = 0
    for x in range(len(d[0])-2):
        #d.sort(d,key = lambda y: y[x])
        ds = np.array(sorted(d, key = lambda y: y[x], reverse=True))
        for r in range(len(ds)):
            split_point = ds[r][x]
            if(r!=len(ds)-1 and ds[r+1][x] == split_point):
                continue
            info_gain = InfoGain(ds, ds[:r+1],ds[r+1:])
            if(info_gain > max_info_gain):
                emitter = x
                value = split_point
                max_info_gain = info_gain        
    return emitter, value

def split(clean_rows):
    Left = []
    Right = []
    em, val = FIND_SPLIT(clean_rows) #splitvalue[0] = emitter and splitvalue[1] = value under that column
    for i in range(len(clean_rows)):
        if(clean_rows[i][em] >= val):
            Left.append(clean_rows[i])
        else:
            Right.append(clean_rows[i])
    return np.array(Left),np.array(Right),em,val

def tree_learning(training_data, depth):
    if(entropy(training_data) == 0):
        label = training_data[0][-1]
        leaf = {'emitter':None, 'value':None, 'room': label, 'right':None, 'left':None}
        return leaf, depth
    else:
        ldata, rdata, em, val = split(training_data)
        root = {'emitter': em, 'value': val, 'room': None, 'right':None, 'left':None}
        root['left'], l_depth = tree_learning(ldata, depth+1)
        root['right'], r_depth = tree_learning(rdata, depth+1)
        return root, max(l_depth, r_depth)

In [3]:
#def prettyprint(tree):
#    datas, *_ = displayData(tree)
#    for data in datas:
#        print(data)

#def displayData(root):
#    if (root['right'] == None) and (root['left'] == None):
#        leaf = '%s' % root['room']
#        w = len(leaf)
#        h = 1
#        m = w // 2
#        return [leaf], w, h, m
#    else:
#        l, a, b, c = displayData(root['left'])
#        r, x, y, z = displayData(root['right'])
#        splittynode = '%s' % root['emitter']
#        k = len(splittynode)
#        line1 = ((c+1)//5)*' ' + ((a-x-1)//5)*'_' + splittynode + (z//5)*'_' + ((x-z)//5)*' '
#        line2 = (c//5)*' ' + '/' + ((a-x-1+k+z)//5)*' ' + '\\' + ((x-z-1)//)*' '
#        if b<y:
#            l += [a*' ']*(y-b)
#        elif b>y:
#            r += [x*' ']*(y-b)
#        zipData = zip(l, r)
#        dataset = [line1, line2] + [it1 + k*' ' + it2 for it1, it2 in zipData]
#        return dataset, a+x+k, max(b, y)+2, a+k // 2

count = [10]

def printP(root):
    pretty(root, 0)

def pretty(tree, space):
    if tree is None:
        return
    else:
        space += (count[0])
        pretty(tree['right'], space)
        print()
        for it in range(count[0], space):
            print(end = ' ')
        if (tree['right'] is None) and (tree['left'] is None):
            print("  ", tree['room'])
        else:
            print("[","x<", tree['value'],',', tree['emitter'],"]")
        pretty(tree['left'], space)

In [ ]:
clean_rows = np.loadtxt("clean_dataset.txt")
t, d = tree_learning(clean_rows, 0)
sys.stdout = open("text.txt", "w")
printP(t)
sys.stdout.close()